# Cтруктура аудитории и доля
Пример расчета соц-дем структуры аудитории тематических каналов.

## Описание задачи и условий расчета
- Период: 01.09.2022 - 30.09.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разибвке по полу, образованию, занятости и группам дохода
- Место просмотра: Все места (дом+дача)
- Каналы: VIJU TV1000, VIJU TV1000 РУССКОЕ, VIJU TV1000 ACTION									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телекомпаний получим id нужных нам каналов
cats.get_tv_company(name='TV1000')

# Это id: 2329, 2334, 3198

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['пол','образование','занятость','группа дохода по России'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: sex, education, work, incomeGroupRussia

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'.
# Минимум 28 дней
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал - минимум 60 минут
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы: TV 1000, TV 1000 ACTION, TV 1000 РУССКОЕ КИНО
company_filter = 'tvCompanyId IN (2329, 2334, 3198)'

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = [
    'tvCompanyName' #Разбиваем по телекомпаниям
]

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 4, #TV Index Plus All Russia
    "totalType": "TotalChannelsThem" #Расчет Share от Измеряемого Тематического. Для расчета от Total Channels поменять на: TotalChannels
}

Сформируем словарь срезов по демографии: ключ - название, значение - entityName демографического атрибута

In [ ]:
demos = {'a. Пол':'sex', 
         'b. Образование':'education', 
         'c. Занятость':'work', 
         'd. Группа дохода по России':'incomeGroupRussia'}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for demo_name, demo_slice in demos.items():
    
    # Подставляем значения словаря в параметры
    project_name = demo_name
    slices.append(demo_slice)
    sortings = {demo_slice:'ASC','tvCompanyName':'ASC'}# отсортируем сначала по демо атрибуту, потом по телекомпании
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name
    tsk['demo_slice'] = demo_slice
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
    #Удаляем демо атрибут из разбивки
    slices.remove(demo_slice)
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}")

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])
    df_result.rename(columns={t['demo_slice']: 'valueName'}, inplace=True) # Меняем название колонки с демо атрибутом для последующего объединения таблиц
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name','valueName']+slices+statistics]

df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - соц-дем переменные, столбцы - каналы, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['prj_name', 'valueName'], 
                        columns=['tvCompanyName'])
df

In [ ]:
# Поменять местами уровни в столбцах (раскомментируйте при необходимости)
#df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

#df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('thematic_01_audience_structure')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)